# Natural Orbital Functional Theory: Dynamic (Scheduled) step size

Import libreries

In [ ]:
import pynof
from scipy.linalg import eigh
import numpy as np
import csv
import sys
# Define energy.py path, in our case is in myCode dir, two directories above
sys.path.insert(1, '../../myCode/')
# read files with optimization functions
from energy import *

Create molecule and choose basis set

In [ ]:
#Molecules Definition
co2 = pynof.molecule("""
0 1
C    0.0000    0.0000    0.0000   
O    1.1000    0.0000    0.0000   
O    -1.1000   -0.0000    0.0000 
""")

bf3 = pynof.molecule("""
0 1
 F     4.007177     0.908823     0.000000
 F     1.724922     0.908844     0.000000
 F     2.866001    -1.067658    -0.000000
 B     2.866001     0.249991    -0.000000
""")

c4h4 = pynof.molecule("""
 0 1
C          0.71430       -0.67320        0.00020
C          0.71480        0.67270       -0.00020
C         -0.71490       -0.67260       -0.00020
C         -0.71420        0.67320        0.00020
H          1.46990       -1.44980        0.00030
H          1.47100        1.44850       -0.00030
H         -1.47110       -1.44850       -0.00030
H         -1.46980        1.44960        0.00020
""")

# Dictionary to facilitate molecule selection with 
# output file name

molecules = {
    "bf3": bf3,
    "co2": co2,
    "c4h4": c4h4,
}

# Select molecule 
molecule = "bf3" # or "co2" or "bf3"
mol = molecules[molecule]

# Select basis set
basis = "cc-pvdz"
p = pynof.param(mol,basis)


Run calculations with a dynamyc alpha value

In [ ]:
# Here we select the NOF functional. 8 should be fine
p.ipnof = 8
p.maxit = 130

p.orb_method="ADAM"

alpha_values = [0.001, 0.002, 0.005, 0.02, 0.04, 0.08, 0.16]
all_energy_data = {}  # Dictionary to store results for each alpha

# File name to store the data
filename = f"{molecule}_{"NOF"}.csv"

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["alpha", "iteration", "energy"])  # Header

    for alpha in alpha_values:
        p.alpha = alpha
        # Select molecule
        _, energy_data = calc_nof_orbrot(mol, p)
        for i_ext, energy in energy_data:
            writer.writerow([alpha, i_ext, energy])

    # ADAM algorithm including dynamic modification, ADAM2
    p.orb_method="ADAM2"
    p.alpha = 0.02
    _, energy_data = calc_nof_orbrot(mol, p)
    for i_ext, energy in energy_data:
        writer.writerow([0.0, i_ext, energy])